In [1]:
import json
import os
from typing import Any, Dict, List, Tuple

In [2]:
def escape_string(value: Any) -> str:
    """Escape single quotes and handle null values for SQL strings."""
    if value is None or value == "":
        return 'null'
    return f"'{str(value).replace("'", "''")}'"

def format_date(date_str: str) -> str:
    """Format date strings, returning 'null' for empty or zero dates."""
    if not date_str or date_str == '0000-00-00 00:00:00' or date_str == '0000-00-00':
        return 'null'
    return f"'{date_str}'"

def format_value(value: Any, value_type: str) -> str:
    """Format value based on its type."""
    if value_type.lower() == 'date':
        return format_date(value)
    elif value_type.lower() == 'str':
        return escape_string(value)
    elif value_type.lower() in ['int', 'bool']:
        return str(value) if value is not None else 'null'
    else:
        return escape_string(value)  # Default to string handling

In [3]:
def convert_json_to_sql(json_data: Dict[str, Any], column_mapping: List[Dict[str, str]], target_tablename:str) -> str:
    """Convert JSON data to SQL insert statement with multiple VALUES."""
    
    # Get column names
    columns = [mapping['target'] for mapping in column_mapping]
    columns_sql = ', '.join(columns)
    
    # Collect all value sets
    all_values = []
    for record in json_data:
        values = []
        for mapping in column_mapping:
            source_field = mapping['source']
            value_type = mapping['type']
            try:
                value = mapping['default']
            except KeyError:
                value = record.get(source_field)    
            formatted_value = format_value(value, value_type)
            values.append(formatted_value)
        all_values.append(f"({', '.join(str(value) for value in values)})")
    
    # Create single INSERT statement with multiple VALUES
    values_sql = ',\n'.join(all_values)
    sql_statement = f"INSERT INTO {target_tablename} ({columns_sql})\nVALUES\n{values_sql};"
    
    return sql_statement

In [ ]:
def main(source_tablename: str, target_tablename: str, data_index: int) -> Tuple[Dict[str, Any], List[Dict[str, str]]]:
    current_dir = os.getcwd()
    parent_dir = os.path.dirname(current_dir)
    assets_dir = os.path.join(parent_dir, '_assets')
    data_filename = 'detoraki_fetusdata_lite.json'
    mapping_filename = 'column_mapping.json'
    
    json_file = os.path.join(assets_dir, data_filename)
    mapping_file = os.path.join(current_dir, mapping_filename)
    

    # Read the input JSON file
    with open(json_file, 'r', encoding='utf-8') as f:
        json_data = json.load(f)
    
    # Read the column mapping file
    with open(mapping_file, 'r', encoding='utf-8') as f:
        column_mapping = json.load(f)
        
    sql_output = convert_json_to_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename)
    print(sql_output)
        

main(source_tablename = 's_org_ext', data_index = 3, target_tablename = 'patients')   
